<a href="https://colab.research.google.com/github/MarquesLazaro/Calculadora-equacao-2-grau/blob/main/janelas_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn as skl
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
import os

lista_csv = os.listdir('/content/drive/MyDrive/Super/Bases Daphnet')
bases = []

for csv in lista_csv:
  with open(f'/content/drive/MyDrive/Super/Bases Daphnet/{csv}', 'r') as arq:
    df = pd.read_csv(arq)
    del df['time_of_sample']

    bases.append(df)

In [ ]:
#64hz -> 64 linhas -> 1s
hz = 64
segundos = 2
sobreposicao = 32
tamanho_janela = hz * segundos

In [ ]:
#Politica de classificação de segmento
def politica_baseline(janela, labels):
  qtde_fog = sum(janela['annotation'])

  if qtde_fog == 0:
    labels.append(0)
    return True
  elif qtde_fog >= tamanho_janela * 0.25:
    labels.append(1)
    return True
  else:
    return False

In [ ]:
janelas = []
labels = []

for base in bases:
  qtde_janelas = base.shape[0] // tamanho_janela
  inicio = 0
  for i in range(qtde_janelas):
    janela = base.iloc[inicio:inicio+tamanho_janela]
    manter_janela = politica_baseline(janela, labels)

    if manter_janela:
      del janela['annotation']
      janelas.append(janela)

    inicio += sobreposicao

labels = np.array(labels)
shape_input = janelas[0].shape[0]

In [ ]:
features = []

for janela in janelas:
  features.append(janela.to_numpy())

features = np.array(features)

print(features.shape)
print(labels.shape)

(14420, 128, 9)
(14420,)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

In [ ]:
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, Activation, Dropout

In [ ]:
inputs = layers.Input(shape=(shape_input, 9))

x = Conv1D(filters=32, kernel_size=6, padding='same')(inputs)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=3)(x)
x = Dropout(0.1)(x)

x = Conv1D(filters=32, kernel_size=6, padding='same')(inputs)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=3)(x)
x = Dropout(0.1)(x)

x = Conv1D(filters=32, kernel_size=6, padding='same')(inputs)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=3)(x)
x = Dropout(0.1)(x)

x = Flatten()(x)
x = Dropout(0.1)(x)
x = Dense(25, activation='relu')(x)
x = Dropout(0.1)(x)

outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 9)]          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 128, 32)           1760      
                                                                 
 activation_2 (Activation)   (None, 128, 32)           0         
                                                                 
 batch_normalization_2 (Bat  (None, 128, 32)           128       
 chNormalization)                                                
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 42, 32)            0         
 g1D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 42, 32)            0     

In [ ]:
callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', verbose=1)]

history = model.fit(x_train,
                    y_train,
                    epochs=30,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=callback)

Epoch 1/30
289/289 [==============================] - 4s 15ms/step - loss: 0.1509 - accuracy: 0.9402 - val_loss: 0.1409 - val_accuracy: 0.9476
Epoch 2/30
289/289 [==============================] - 4s 13ms/step - loss: 0.1380 - accuracy: 0.9486 - val_loss: 0.1324 - val_accuracy: 0.9484
Epoch 3/30
289/289 [==============================] - 3s 11ms/step - loss: 0.1270 - accuracy: 0.9519 - val_loss: 0.1267 - val_accuracy: 0.9519
Epoch 4/30
289/289 [==============================] - 3s 11ms/step - loss: 0.1216 - accuracy: 0.9534 - val_loss: 0.1256 - val_accuracy: 0.9515
Epoch 5/30
289/289 [==============================] - 4s 14ms/step - loss: 0.1188 - accuracy: 0.9552 - val_loss: 0.1265 - val_accuracy: 0.9549
Epoch 6/30
289/289 [==============================] - 4s 14ms/step - loss: 0.1135 - accuracy: 0.9548 - val_loss: 0.1218 - val_accuracy: 0.9528
Epoch 7/30
289/289 [==============================] - 3s 11ms/step - loss: 0.1059 - accuracy: 0.9594 - val_loss: 0.1231 - val_accuracy: 0.9545

In [ ]:
loss = model.evaluate(x_test, y_test)

91/91 [==============================] - 0s 5ms/step - loss: 0.1554 - accuracy: 0.9424


In [ ]:
y_pred_prob = model.predict(x_test)
y_pred = (y_pred_prob > 0.5).astype(int)
y_test = (y_test > 0.5).astype(int)

91/91 [==============================] - 0s 3ms/step


In [ ]:
cm = skl.metrics.confusion_matrix(y_test, y_pred)
cm

array([[2478,   44],
       [ 122,  240]])

In [ ]:
report = skl.metrics.classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.98      0.97      2522
           1       0.85      0.66      0.74       362

    accuracy                           0.94      2884
   macro avg       0.90      0.82      0.86      2884
weighted avg       0.94      0.94      0.94      2884

